In [1]:
import datetime
import pandas as pd
import json
import os
import openai

In [3]:
# Have file with the key and organization: {"api_key": "x", "organization_id": "x"}
with open("../keys/key.json", 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)
 

In [4]:
openai.api_key = json_object["api_key"]
openai.organization = json_object["organization"]

def get_response(system_message, user_message, max_tokens=256):
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-16k",
      messages=[
        {
          "role": "system",
          "content": system_message
        },
        {
          "role": "user",
          "content": user_message
        }
      ],
      temperature=0.01,
      max_tokens=max_tokens
    )
    return response




In [5]:
get_response("Get the sentiments of the sentence and return as a json", "This services is amazing")

<OpenAIObject chat.completion id=chatcmpl-7ktxGjdbFmwu6OCAEPYFOBC2D8h2M at 0x7f653057bd10> JSON: {
  "id": "chatcmpl-7ktxGjdbFmwu6OCAEPYFOBC2D8h2M",
  "object": "chat.completion",
  "created": 1691412534,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"sentiment\": \"positive\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 26,
    "completion_tokens": 10,
    "total_tokens": 36
  }
}

In [6]:
# Open files
filename = '../data/hrmc_feedback.csv'
with open(filename, 'r') as file:
    # Reading the entire content of the file
    content = file.read()
    

In [7]:
import pandas as pd
df = pd.read_csv(filename)
df.head()

,Unnamed: 0,path_or_service_name,feedback
0,0,/government/publications/vat-registration-noti...,address for VAT1c\naddrss on notes diffrent to...
1,1,/guidance/register-for-vat-if-you-own-land-wit...,Trying to print how to register for VAT when o...
2,2,/guidance/register-for-vat-if-you-own-land-wit...,reading the notice\nThe notice does not say wh...
3,3,/government/publications/vat-application-for-r...,Viewing VAT 1B Notes\nAddress for [REMOVED] on...
4,4,/guidance/check-when-you-can-expect-a-reply-fr...,want to send you a letter but no address\nno a...


In [8]:
# Newlines might confuse AI reading the csv data.
df["feedback"] = df["feedback"].str.replace("\n", " ")


In [ ]:
df_test = df.copy()
df_test["what_where_you_doing"] = [feedback.split("\n", 1)[0] for feedback in df.feedback]
df_test["what_went_wrong"]= [feedback.split("\n", 1)[1] for feedback in df.feedback]
df_test

In [ ]:
df = df.rename(columns = {df.keys()[0]: "key"})


In [ ]:
system_message = "You will parse the data as a csv, \
                    and extract the following data \
                    key: \
                    service: \
                    topic: topic of feedback, \
                    sumary: no more than a 10 word summary of the feedback, \
                    problem: boolean whether people have a problem, \
                    problem_summary: a few word summary of the problem, \
                    website_issue: boolean whether it relates to website issues \
                    tax_issue: bollean technical tax issue, \
                    guidance_issue: boolean whether it relates to guidance issues, \
                    spam: boolean is it spam, \
                    sentiment: the sentiment of the feedback , i.e. positive, negative, neurtral, \
                    language: the language of the feedback, e.g. English, French \
                    fraud: boolean does it relate to fraud, \
                    urgency: boolean is it urgent, \
                    seriousness: boolean is it serious, \
                    keywords: and keywords of the feedback \
                    in a json format"

responses = []
for i in range(len(df)//5):
    user_message = df[(5*i):(5 + 5*i)].to_csv()
    resp = get_response(system_message, user_message , max_tokens=10000)
    responses.append(resp.choices[0].message.content)


In [ ]:
LLM_output = pd.concat([pd.read_json(response) for response in responses])
sample_df = pd.concat([df, LLM_output.reset_index()], axis =1)[0:len(LLM_output)]
sample_df

In [ ]:
today = datetime.datetime.today()
start_day = today + datetime.timedelta(days=-390)
sample_df["date"] = [(start_day + datetime.timedelta(days=row/30)).strftime('%Y-%m-%d') for row in range(0, len(sample_df))]



In [ ]:
sample_df.to_csv("../data/feedback_output_v3.csv")